In [1]:
from llama_cpp import Llama
from transformers import AutoTokenizer

In [2]:
model_path='./mbti-senti_model'

tokenizer = AutoTokenizer.from_pretrained(model_path)

model = Llama(
    model_path='./mbti-senti_model/unsloth.Q4_K_M.gguf', #다운로드받은 모델의 위치
    n_ctx=1024,
    n_gpu_layers=33        # Number of model layers to offload to GPU
)

llama_model_loader: loaded meta data with 25 key-value pairs and 291 tensors from ./mbti-senti_model/unsloth.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mbti Model
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                       llama.context_length u32              = 8192
llama_model_loader: - kv   6:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   7:                  llama.feed_forward_len

In [3]:
import pandas as pd

df = pd.read_csv('./senti.csv', encoding='UTF-8')

sampled_df = df.sample(frac=0.2, random_state=42)



texts  = list(sampled_df['text'])
sentis = list(sampled_df['senti'])

sentis

['행복',
 '놀람',
 '놀람',
 '혐오',
 '놀람',
 '분노',
 '분노',
 '중립',
 '분노',
 '공포',
 '행복',
 '혐오',
 '행복',
 '공포',
 '놀람',
 '행복',
 '슬픔',
 '행복',
 '공포',
 '혐오',
 '행복',
 '혐오',
 '중립',
 '놀람',
 '공포',
 '행복',
 '분노',
 '혐오',
 '슬픔',
 '슬픔',
 '분노',
 '분노',
 '중립',
 '행복',
 '행복',
 '슬픔',
 '중립',
 '행복',
 '중립',
 '분노',
 '행복',
 '중립',
 '분노',
 '행복',
 '혐오',
 '행복',
 '중립',
 '놀람',
 '중립',
 '분노',
 '행복',
 '놀람',
 '행복',
 '놀람',
 '분노',
 '행복',
 '중립',
 '공포',
 '슬픔',
 '행복',
 '놀람',
 '분노',
 '분노',
 '행복',
 '혐오',
 '놀람',
 '중립',
 '행복',
 '분노',
 '중립',
 '중립',
 '행복',
 '혐오',
 '놀람',
 '행복',
 '분노',
 '행복',
 '중립',
 '행복',
 '혐오',
 '슬픔',
 '혐오',
 '중립',
 '중립',
 '놀람',
 '행복',
 '행복',
 '행복',
 '공포',
 '중립',
 '슬픔',
 '분노',
 '중립',
 '행복',
 '혐오',
 '행복',
 '공포',
 '중립',
 '중립',
 '공포',
 '분노',
 '중립',
 '혐오',
 '행복',
 '분노',
 '공포',
 '놀람',
 '혐오',
 '행복',
 '행복',
 '혐오',
 '중립',
 '중립',
 '중립',
 '슬픔',
 '분노',
 '슬픔',
 '중립',
 '분노',
 '분노',
 '중립',
 '행복',
 '행복',
 '분노',
 '행복',
 '분노',
 '행복',
 '행복',
 '행복',
 '중립',
 '중립',
 '행복',
 '행복',
 '분노',
 '행복',
 '행복',
 '분노',
 '중립',
 '분노',
 '슬픔',
 '슬픔',
 '공포',
 '분노',

In [4]:
output_sentis = []

for text in texts:

    PROMPT = "정확한 챗봇으로서 상대방의 입력에 대해 감정을 맞추자. 모든 대답은 '행복', '분노', '슬픔', '중립', '혐오', '놀람', '공포' 중 하나로 대답해줘."
    instruction = text

    messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    generation_kwargs = {
        "max_tokens": 2048,
        "stop": ["<|eot_id|>"],
        "top_p": 0.9,
        "temperature": 0.6,
        "echo": True,  # 프롬프트를 출력에 포함합니다.
    }

    response_msg = model(prompt, **generation_kwargs)
    output_senti = response_msg['choices'][0]['text'][len(prompt):]

    output_sentis.append(output_senti)

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1129: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(

llama_print_timings:        load time =     116.48 ms
llama_print_timings:      sample time =       0.16 ms /     3 runs   (    0.05 ms per token, 18633.54 tokens per second)
llama_print_timings: prompt eval time =     116.37 ms /    90 tokens (    1.29 ms per token,   773.40 tokens per second)
llama_print_timings:        eval time =      32.63 ms /     2 runs   (   16.31 ms per token,    61.30 tokens per second)
llama_print_timings:       total time =     149.93 ms /    92 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     116.48 ms
llama_print_timings:      sample time =       0.21 ms /     4 runs   (    0.05 ms per token, 18867.92 tokens per second)
llama_print_timings: prompt eval time =      29.36 ms /    25 tokens (    1.17 ms 

In [5]:
output_sentis

['행복',
 '놀람',
 '놀람',
 '슬픔',
 '놀람',
 '분노',
 '분노',
 '중립',
 '행복',
 '슬픔',
 '행복',
 '혐오',
 '행복',
 '슬픔',
 '놀람',
 '행복',
 '슬픔',
 '행복',
 '공포',
 '슬픔',
 '행복',
 '분노',
 '중립',
 '놀람',
 '놀람',
 '행복',
 '분노',
 '혐오',
 '슬픔',
 '행복',
 '분노',
 '분노',
 '중립',
 '행복',
 '행복',
 '슬픔',
 '중립',
 '행복',
 '중립',
 '분노',
 '행복',
 '중립',
 '분노',
 '행복',
 '혐오',
 '행복',
 '놀람',
 '놀람',
 '중립',
 '행복',
 '행복',
 '행복',
 '행복',
 '놀람',
 '슬픔',
 '행복',
 '행복',
 '놀람',
 '슬픔',
 '행복',
 '중립',
 '중립',
 '슬픔',
 '행복',
 '분노',
 '놀람',
 '중립',
 '슬픔',
 '분노',
 '중립',
 '중립',
 '행복',
 '행복',
 '놀람',
 '행복',
 '분노',
 '슬픔',
 '중립',
 '행복',
 '분노',
 '슬픔',
 '혐오',
 '행복',
 '중립',
 '놀람',
 '행복',
 '행복',
 '행복',
 '슬픔',
 '행복',
 '슬픔',
 '분노',
 '중립',
 '슬픔',
 '행복',
 '중립',
 '공포',
 '중립',
 '행복',
 '중립',
 '행복',
 '슬픔',
 '혐오',
 '행복',
 '분노',
 '공포',
 '놀람',
 '분노',
 '중립',
 '행복',
 '혐오',
 '행복',
 '중립',
 '중립',
 '분노',
 '중립',
 '슬픔',
 '혐오',
 '분노',
 '행복',
 '놀람',
 '행복',
 '행복',
 '슬픔',
 '행복',
 '행복',
 '행복',
 '행복',
 '행복',
 '중립',
 '중립',
 '행복',
 '슬픔',
 '분노',
 '중립',
 '행복',
 '행복',
 '중립',
 '슬픔',
 '슬픔',
 '슬픔',
 '공포',
 '슬픔',

In [6]:
from sklearn.metrics import precision_recall_fscore_support

In [7]:
precision, recall, f1, _ = precision_recall_fscore_support(sentis, output_sentis, labels=['행복', '분노', '슬픔', '중립', '혐오', '놀람', '공포'])

In [8]:
import pandas as pd

metrics_df = pd.DataFrame({
    'Class': ['행복', '분노', '슬픔', '중립', '혐오', '놀람', '공포'],
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
})

metrics_df

,Class,Precision,Recall,F1 Score
0,행복,0.701170,0.820864,0.756311
1,분노,0.628521,0.545038,0.583810
2,슬픔,0.538244,0.716981,0.614887
3,중립,0.680927,0.602524,0.639331
4,혐오,0.820580,0.668817,0.736967
5,놀람,0.777487,0.822715,0.799462
6,공포,0.993976,0.587189,0.738255


In [9]:
from sklearn.metrics import accuracy_score

In [10]:
accuracy = accuracy_score(sentis, output_sentis)
accuracy

0.6892903225806452

In [11]:
len(sentis)

3875

: 